In [1]:
import re
import os
import pickle
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from _html_parser import HtmlParser, ParsingDataPrepare

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)

In [2]:
%load_ext autoreload
%autoreload 2

### All teams data - new solution with work with 'full_data' file.

In [3]:
# with open('pickle_files/teams_data/df_full_teams_data_cr_11_10_2022', 'rb') as f:
#     df_full_teams_data_cr_11_10_2022 = pickle.load(f)

In [4]:
# print('shape - ', df_full_teams_data_cr_11_10_2022.shape)
# df_full_teams_data_cr_11_10_2022.tail()

In [5]:
# path = 'pickle_files/teams_data/tmd_16_09_2022/'
# soccerway_tmd_16_09_2022 = []

# for file in os.listdir(path):
#     if file.startswith('soccerway_team_data'):
#         with open(path + file, 'rb') as f:
#             tm_data = pickle.load(f)
#             soccerway_tmd_16_09_2022 = soccerway_tmd_16_09_2022 + tm_data

# print('len - ', len(soccerway_tmd_16_09_2022))        

In [6]:
# path = 'pickle_files/teams_data/tmd_26_03_2021/'
# soccerway_tmd_26_03_2021 = []

# for file in os.listdir(path):
#     if file.startswith('soccerway_team_data'):
#         with open(path + file, 'rb') as f:
#             tm_data = pickle.load(f)
#             soccerway_tmd_26_03_2021 = soccerway_tmd_26_03_2021 + tm_data

# print('len - ', len(soccerway_tmd_26_03_2021))        

In [7]:
# path = 'pickle_files/teams_data/tmd_02_11_2013/'
# soccerway_tmd_02_11_2013 = []

# for file in os.listdir(path):
#     if file.startswith('soccerway_team_data'):
#         with open(path + file, 'rb') as f:
#             tm_data = pickle.load(f)
#             soccerway_tmd_02_11_2013 = soccerway_tmd_02_11_2013 + tm_data

# print('len - ', len(soccerway_tmd_02_11_2013))        

In [8]:
# path = 'pickle_files/teams_data/tmd_09_07_2005/'
# soccerway_tmd_09_07_2005 = []

# for file in os.listdir(path):
#     if file.startswith('soccerway_team_data'):
#         with open(path + file, 'rb') as f:
#             tm_data = pickle.load(f)
#             soccerway_tmd_09_07_2005 = soccerway_tmd_09_07_2005 + tm_data

# print('len - ', len(soccerway_tmd_09_07_2005))        

In [9]:
# soccerway_tmd_exist = soccerway_tmd_16_09_2022 + soccerway_tmd_26_03_2021 + soccerway_tmd_02_11_2013 + soccerway_tmd_09_07_2005
# print('len - ', len(soccerway_tmd_exist))
# soccerway_tmd_exist[0]

In [10]:
# df_soccerway_tmd_exist = pd.DataFrame.from_dict(soccerway_tmd_exist)
# print('shape - ', df_soccerway_tmd_exist.shape)
# df_soccerway_tmd_exist.rename(columns={0:'f_team', 1: 'country', 2: 'data'}, inplace=True)
# df_soccerway_tmd_exist.tail(2)

In [11]:
# df_soccerway_tmd_exist.drop_duplicates(subset=['f_team', 'country'], inplace=True)
# print('shape - ', df_soccerway_tmd_exist.shape)

#### Change different languages columns names for English

In [12]:
# data_lt_exist = df_soccerway_tmd_exist.data.to_list()
# data_lt_exist = [{'Team Name': 'nd'} if x == 'nd' else x for x in data_lt_exist]

# for val in data_lt_exist:
#     for key in list(val.keys()):
#         if key in ['Adresse', 'Adresa', 'Alamat']:
#             val['Address'] = val.pop(key)
#         elif key in ['Land', 'Ţara', 'Negara']:
#             val['Country'] = val.pop(key)
#         elif key in ['Stadt', 'Oraş', 'Kota']:
#             val['City'] = val.pop(key)
#         elif key in ['Zuschauer', 'Capacitate', 'Kapasitas']:
#             val['Capacity'] = val.pop(key)    
#         elif key in ['Fondat', '', 'Didirikan']:
#             val['Founded'] = val.pop(key)  
#         elif key in ['Telefon', 'Telepon']:
#             val['Phone'] = val.pop(key)
#         elif key in ['Nume', 'Nama']:
#             val['Name'] = val.pop(key) 

#### ****************************************************************

In [13]:
# df_team_country_data_exist = pd.DataFrame.from_records(data_lt_exist)
# df_team_country_data_exist.tail(1)

In [14]:
# df_soccerway_tmd_exist.reset_index(drop=True, inplace=True)
# df_soccerway_tmd_exist.tail(1)

In [15]:
# print('shape - ', df_team_country_data_exist.shape)
# print('shape - ', df_soccerway_tmd_exist.shape)

In [16]:
# df_tmd_exist = df_soccerway_tmd_exist[['f_team', 'country']].merge(df_team_country_data_exist, how ='left', left_index=True, right_index=True)
# print('shape - ', df_soccerway_tmd_exist.shape)

In [17]:
# df_tmd_exist.rename(columns={'Country': 'country_prsng'}, inplace=True)

In [18]:
# df_tmd_exist.columns = df_tmd_exist.columns.str.lower()

In [19]:
# df_tmd_exist.tail()

In [20]:
# df_tmd_not_exist = df_soccerway_tmd_exist[df_soccerway_tmd_exist.f_team.isin(df_tmd_main_created_18_09_2022.f_team)]

In [21]:
# df_tmd_not_exist = df_tmd_main_created_18_09_2022[~df_tmd_main_created_18_09_2022.set_index(['f_team', 'country'])
#                                                                                  .index.isin(df_soccerway_tmd_exist.set_index(['f_team', 'country']).index)]

In [22]:
# print('shpe - ', df_tmd_not_exist.shape)
# df_tmd_not_exist.tail(3)

In [23]:
# teams_countries_lt = df_tmd_not_exist[['f_team', 'country']].values.tolist()

### Parsing teams data if don't exist:

In [24]:
with open('pickle_files/teams_data/tmd_bets_coef_w_events/df_teams_cities_na_bets_coef_with_events', 'rb') as f:
    df_teams_na_cities = pickle.load(f)

#### Parsing teams ids from soccer365

In [25]:
# teams_for_ids = df_teams_na_cities.f_team.values
# print('shape - ', len(teams_for_ids))
# teams_for_ids[-3:]

In [26]:
# HtmlParser.find_teams_ids(teams_for_ids[:]) 

In [27]:
# with open('pickle_files/teams_data/tmd_26_03_2021/teams_ids_na_cities_Waltersdorf', 'rb') as f:
#     lst_teams_ids = pickle.load(f)

In [28]:
# print('shape - ', len(lst_teams_ids))
# lst_teams_ids[-3:]

In [29]:
# teams_ids = [x[1] for x in lst_teams_ids if x[1] is not None]

In [30]:
# print('shape - ', len(teams_ids))
# teams_ids[-3:]

In [31]:
# lst_teams_wt_id = [x[0] for x in lst_teams_ids if x[1] is None]

In [32]:
# print('shape - ', len(lst_teams_wt_id))
# lst_teams_wt_id[-3:]

In [33]:
# df_teams_ids           = pd.DataFrame(columns=['teams_ids', 'team'])
# df_teams_ids.teams_ids = [x[1] for x in lst_teams_ids]
# df_teams_ids.team      = [x[0] for x in lst_teams_ids]

In [34]:
# df_teams_ids = df_teams_ids.merge(df_game_ids_na_cities.country, how='left', left_index=True, right_index=True)

In [35]:
# df_teams_ids.head(3)

#### Parsing teams data from soccer365

In [36]:
# HtmlParser.find_teams_data(teams_ids[:])

In [37]:
# tmd_26_05_2022: (teams_wiki_data_8636, teams_wiki_data_12683, teams_wiki_data_14892), 
# with open('pickle_files/teams_data/tmd_26_03_2021/teams_wiki_data_7887', 'rb') as f:
#     wiki_data = pickle.load(f)

In [38]:
# len(wiki_data)

In [39]:
# wiki_data[2:3]

In [40]:
# df_wiki_data  = pd.DataFrame(wiki_data, columns=['teams_ids', 'tm_data'])
# df_teams_data = df_teams_ids.merge(df_wiki_data, how='left', left_on='teams_ids', right_on='teams_ids')

In [41]:
# df_teams_data.drop_duplicates(subset=['teams_ids', 'team'], inplace=True)
# df_teams_data.reset_index(drop=True, inplace=True)

In [42]:
# print('shape - ', df_teams_data.shape)
# df_teams_data.head()

#### Data clear soccer365

In [43]:
# data_is_a = df_teams_data.tm_data.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'is a', '\.')) if re.findall('is a', str(x)) else None)

In [44]:
# data_based_in = data_is_a.map(lambda x: x if re.findall('(based in)|(city of)|(town of)|(located in)|(club from)|(club in)|(playing in)', str(x)) else None)

In [45]:
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'city of ', '$')) if re.findall('city of', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'town of ', '$')) if re.findall('town of', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'located in ', '$')) if re.findall('located in', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'club from ', '$')) if re.findall('club from', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'club in ', '$')) if re.findall('club in', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'playing in ', '$')) if re.findall('playing in', str(x)) else x)
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, 'based in ', '$')) if re.findall('based in', str(x)) else x)

In [46]:
# data_based_in = data_based_in.map(lambda x: ''.join(HtmlParser.cut_part_of_string(x, '^', ',')) if re.findall(',', str(x)) else x)

In [47]:
## data_based_in

In [48]:
# df_teams_data['city'] = data_based_in.values

In [49]:
# print('shape - ', df_teams_data.shape[0])
# df_teams_data.head(3)

In [50]:
# df_teams_data.to_csv('csv_files/df_teams_data_26_03_2021_sec.csv', index = None, header=True)

### Parsing data from soccerway

In [51]:
teams_countries_lt = df_teams_na_cities.values.tolist()

In [52]:
len(teams_countries_lt) 

1972

In [53]:
teams_countries_lt[:3]

[['Botafogo Paraíba', 'Brazil'],
 ['Galway United', 'Republic of Ireland'],
 ['Costa del Este', 'Panama']]

In [54]:
unique_tc = [tuple(x) for x in teams_countries_lt]
len(set(unique_tc))

1972

In [55]:
# 1 Stop *****************************************************************

##### Google can block ip address as bot's requests

In [56]:
# HtmlParser.find_google_soccerway_team_urls(teams_countries_lt, name_saved_file='soccerway_team_urls_tmd_bets_coef_w_events')

In [57]:
with open('pickle_files/teams_data/tmd_bets_coef_w_events/soccerway_team_urls_tmd_bets_coef_w_events_Mozyr NPZ_Belarus', 'rb') as f:
    teams_urls_lt = pickle.load(f)  

In [58]:
print('len - ', len(teams_urls_lt))
teams_urls_lt[:1]

len -  1972


[['Botafogo Paraíba',
  'Brazil',
  'https://int.soccerway.com/teams/brazil/botafogo-futebol-clube-joao-pessoa/6112/']]

In [59]:
df_urls = pd.DataFrame({'team':[x[0] for x in teams_urls_lt], 'country':[x[1] for x in teams_urls_lt], 'urls':[x[2] for x in teams_urls_lt]})

In [60]:
print('shape - ', df_urls.shape)
df_urls.tail(2)

shape -  (1972, 3)


,team,country,urls
1970,Bobovnya,Belarus,https://int.soccerway.com/teams/belarus/bobovnya/58363/
1971,Mozyr NPZ,Belarus,https://int.soccerway.com/teams/belarus/mozyr-npz/53411/


In [61]:
df_urls.urls = df_urls.urls.map(lambda x: re.sub( '[a-z]*/$', '', x))

#### Some urls have 'ru', 'ro' soccerway path for not English language. Fix it - columns have not English names and merge separately.

In [62]:
df_urls.urls = df_urls.urls.map(lambda x: re.sub('://ru', '://int', x))
df_urls.urls = df_urls.urls.map(lambda x: re.sub('://ro', '://int', x))

In [63]:
ru_urls_lt = df_urls.urls.map(lambda x: x if ''.join(HtmlParser.cut_part_of_string(x, 'https://', '.soccerway.com')) == 'ru' else None).to_list()
set(ru_urls_lt)

{None}

##### Some urls have not soccerway team path and too cut

In [64]:
sr_not_soccerway_teams_urls = df_urls.urls.map(lambda x: x if ''.join(HtmlParser.cut_part_of_string(x, 'soccerway.com/', '/')) != 'teams' else None)

In [65]:
sr_not_soccerway_teams_urls[~sr_not_soccerway_teams_urls.isna()]

206                                                  https://www.sofascore.com/team/football/tj-sokol-libis/207460
470     https://int.soccerway.com/matches/2019/05/15/nigeria/premier-league/kada-city/akwa-united-fc/2967444/head2
620                     https://int.soccerway.com/matches/2018/09/08/slovenia/3-snl/plama-podgrad/fc-koper/2885713
1085                                                                         https://en.wikipedia.org/wiki/Kirkley
1188                                                 https://www.fragrantica.ua/perfumes/Byredo/Sellier-58572.html
1250                                                        https://int.soccerway.com/players/vasilije-baki/464187
1348                                                      https://developer.mozilla.org/ru/docs/Web/HTML/Element/A
1455                                                                                           https://sport.ua/uk
Name: urls, dtype: object

In [66]:
sr_not_soccerway_teams_inds = sr_not_soccerway_teams_urls[~sr_not_soccerway_teams_urls.isna()].index.tolist()
str(sr_not_soccerway_teams_inds)

'[206, 470, 620, 1085, 1188, 1250, 1348, 1455]'

In [67]:
df_not_soccerway_teams = df_urls[['team', 'country']][df_urls.index.isin(sr_not_soccerway_teams_inds)]

In [68]:
## If 'not_soccerway_teams' is empty:
# not_soccerway_teams = []
## Else:
not_soccerway_teams_lt = df_not_soccerway_teams.apply(lambda x: (x[0], x[1]), axis=1).tolist()
print('len - ', len(not_soccerway_teams_lt))
not_soccerway_teams_lt[:3]

len -  8


[('Sokol Libis', 'Czech Republic'),
 ('Kada City', 'Nigeria'),
 ('Plama Podgrad', 'Slovenia')]

In [69]:
# Error urls haven't '/' or any number in the end of string
error_urls_lt = df_urls.urls.map(lambda x: None if re.findall(r'(\d|/)$', x) else x)
error_urls_lt[~error_urls_lt.isna()]

1085                            https://en.wikipedia.org/wiki/Kirkley
1188    https://www.fragrantica.ua/perfumes/Byredo/Sellier-58572.html
1348         https://developer.mozilla.org/ru/docs/Web/HTML/Element/A
1455                                              https://sport.ua/uk
1766         https://www.soccerway.com/teams/france/olympique-saumur-
1913                    https://int.soccerway.com/teams/austria/lask-
Name: urls, dtype: object

In [70]:
error_urls_lt_inds = error_urls_lt[~error_urls_lt.isna()].index.tolist()
str(error_urls_lt_inds)

'[1085, 1188, 1348, 1455, 1766, 1913]'

In [71]:
df_error_urls = df_urls[['team', 'country']][df_urls.index.isin(error_urls_lt_inds)]
df_error_urls.head(3)

,team,country
1085,Kirkley & Pakefield,England
1188,Sellier & Bellot Vlašim,Colombia
1348,A&N,Republic of Kosovo


In [72]:
## If 'df_error_urls' is empty:
# error_urls_teams_lt = []
## Else:
error_urls_teams_lt = df_error_urls.apply(lambda x: (x[0], x[1]), axis=1).tolist()
print('len - ', len(error_urls_teams_lt))
error_urls_teams_lt[:3]

len -  6


[('Kirkley & Pakefield', 'England'),
 ('Sellier & Bellot Vlašim', 'Colombia'),
 ('A&N', 'Republic of Kosovo')]

In [73]:
error_urls_names_lt = list(set(not_soccerway_teams_lt + error_urls_teams_lt))
print('len - ', len(error_urls_names_lt))
str(error_urls_names_lt)

len -  10


"[('Kirkley & Pakefield', 'England'), ('Sport & Leisure', 'Northern Ireland'), ('A&N', 'Republic of Kosovo'), ('Semendria', 'Serbia'), ('Olympique Saumur', 'France'), ('LASK Amateure', 'Austria'), ('Sokol Libis', 'Czech Republic'), ('Sellier & Bellot Vlašim', 'Colombia'), ('Kada City', 'Nigeria'), ('Plama Podgrad', 'Slovenia')]"

In [74]:
error_urls_inds_lt = list(set(sr_not_soccerway_teams_inds + error_urls_lt_inds))
print('len - ', len(error_urls_inds_lt))
str(error_urls_inds_lt)

len -  10


'[1250, 1188, 1348, 1766, 620, 206, 1455, 470, 1913, 1085]'

In [75]:
df_corr_urls = df_urls[~df_urls.index.isin(error_urls_inds_lt)].copy()
print('shape -', df_corr_urls.shape)
df_corr_urls.tail(3)

shape - (1962, 3)


,team,country,urls
1969,St. Louis City II,USA,https://int.soccerway.com/teams/united-states/st-louis-city-ii/58082
1970,Bobovnya,Belarus,https://int.soccerway.com/teams/belarus/bobovnya/58363
1971,Mozyr NPZ,Belarus,https://int.soccerway.com/teams/belarus/mozyr-npz/53411


In [76]:
teams_urls_lt = df_corr_urls.apply(lambda x: list(x), axis=1).to_list()
teams_urls_lt[:2]

[['Botafogo Paraíba',
  'Brazil',
  'https://int.soccerway.com/teams/brazil/botafogo-futebol-clube-joao-pessoa/6112'],
 ['Galway United',
  'Republic of Ireland',
  'https://int.soccerway.com/teams/ireland-republic/galway/26455']]

In [77]:
# 2 Stop **********************************************************************

#### Some urls can't exist already and in teams_data_lt data == 'nd'

In [78]:
# HtmlParser.find_teams_data_soccerway(teams_urls_lt, name_saved_file = 'soccerway_team_data_bets_coef_w_events')

In [79]:
with open('pickle_files/teams_data/tmd_bets_coef_w_events/soccerway_team_data_bets_coef_w_events_Mozyr NPZ_Belarus', 'rb') as f:
    teams_data_lt = pickle.load(f)   

In [80]:
len(teams_data_lt)

1962

##### Change as 'nd' some errors urls with lang names from parsing list

In [81]:
inds = []
for ind, x in enumerate(teams_data_lt):
    if type(x[2]) is dict:
        for key in x[2].keys():
            if re.findall(r'[^.a-z A-Z-]', key):
                inds.append(ind)
inds = set(inds)  
inds

{509}

In [82]:
for ind in inds:
    teams_data_lt[ind][2] = 'nd'
    print(teams_data_lt[ind])

['Kiwi', 'Brazil', 'nd']


##### ******************************************

In [83]:
nd_data_lt = [(x[0], x[1]) for x in teams_data_lt if x[2] == 'nd']
print('len - ', len(nd_data_lt))
nd_data_lt

len -  5


[('Maccabi Yavne', 'Israel'),
 ('Kiwi', 'Brazil'),
 ('FC Hatvan', 'Hungary'),
 ('Bethune', 'France'),
 ('1910 Oradea', 'Romania')]

#### Concatinate error urls team-country names with urls after parsing with 'nd' data and 'ids' too.

In [84]:
error_urls_names_nd_data_lt = list(set(error_urls_names_lt + nd_data_lt))
print('len - ', len(error_urls_names_nd_data_lt))
str(error_urls_names_nd_data_lt)

len -  15


"[('Kiwi', 'Brazil'), ('1910 Oradea', 'Romania'), ('Sport & Leisure', 'Northern Ireland'), ('Kirkley & Pakefield', 'England'), ('A&N', 'Republic of Kosovo'), ('Semendria', 'Serbia'), ('Olympique Saumur', 'France'), ('Bethune', 'France'), ('Sokol Libis', 'Czech Republic'), ('Sellier & Bellot Vlašim', 'Colombia'), ('FC Hatvan', 'Hungary'), ('Kada City', 'Nigeria'), ('Maccabi Yavne', 'Israel'), ('LASK Amateure', 'Austria'), ('Plama Podgrad', 'Slovenia')]"

In [85]:
df_nd_data_inds = df_urls[['team', 'country']].apply(lambda x: x.index if (x[0], x[1]) in nd_data_lt else None, axis=1)

In [86]:
nd_data_df_urls_inds = df_nd_data_inds[~df_nd_data_inds.isna()].index.tolist()
print('len - ', len(nd_data_df_urls_inds))
nd_data_df_urls_inds

len -  5


[64, 511, 866, 1060, 1475]

In [87]:
df_corr_urls[df_corr_urls.index.isin(nd_data_df_urls_inds)]

,team,country,urls
64,Maccabi Yavne,Israel,https://int.soccerway.com/teams/israel/maccabi-yavne/3092
511,Kiwi,Brazil,https://br.soccerway.com/teams/samoa/kiwi-fc/17403
866,FC Hatvan,Hungary,https://int.soccerway.com/teams/hungary/fc-hatvan/19696
1060,Bethune,France,https://int.soccerway.com/teams/france/bethune/32285/
1475,1910 Oradea,Romania,https://ca.soccerway.com/teams/romania/1910-oradea/51352


In [88]:
error_urls_nd_data_inds_lt = list(set(error_urls_inds_lt + nd_data_df_urls_inds))
print('len - ', len(error_urls_nd_data_inds_lt))
str(error_urls_nd_data_inds_lt)

len -  15


'[64, 1250, 866, 1188, 1348, 1766, 1060, 1475, 620, 206, 1455, 470, 1913, 1085, 511]'

#### Remove 'nd' data index from 'df_corr_urls'

In [89]:
df_corr_urls = df_corr_urls[~df_corr_urls.index.isin(nd_data_df_urls_inds)].copy()
print('shape -', df_corr_urls.shape)
df_corr_urls.tail(3)

shape - (1957, 3)


,team,country,urls
1969,St. Louis City II,USA,https://int.soccerway.com/teams/united-states/st-louis-city-ii/58082
1970,Bobovnya,Belarus,https://int.soccerway.com/teams/belarus/bobovnya/58363
1971,Mozyr NPZ,Belarus,https://int.soccerway.com/teams/belarus/mozyr-npz/53411


##### Remove index with 'nd' from 'teams_data_lt'

In [90]:
teams_with_data_lt = [x for x in teams_data_lt if (x[0], x[1]) not in nd_data_lt]

In [91]:
len(teams_with_data_lt)

1957

In [92]:
df_team_country = pd.DataFrame({'team': [x[0] for x in teams_with_data_lt], 'country': [x[1] for x in teams_with_data_lt]})
print('shape - ', df_team_country.shape)
df_team_country.tail(2)

shape -  (1957, 2)


,team,country
1955,Bobovnya,Belarus
1956,Mozyr NPZ,Belarus


In [93]:
data_lt = [x[2] for x in teams_with_data_lt]
str(data_lt[:1])

"[{'Team Name': 'Botafogo FC (João Pessoa)', 'Founded': '1931', 'Address': 'Maravilha do Contorno, Bairro Cristo Redentor 58010-000 João Pessoa, Paraíba', 'Country': 'Brazil', 'Phone': '+55 (83) 231 5547', 'Name': 'Estádio José Américo de Almeida Filho', 'City': 'João Pessoa, Paraíba', 'Capacity': '25770'}]"

In [94]:
df_team_data = pd.DataFrame.from_dict(data_lt)
print('shape - ', df_team_data.shape)
df_team_data.tail(2)

shape -  (1957, 10)


,Team Name,Founded,Address,Country,Phone,Name,City,Capacity,E-mail,Fax
1955,FK Bobovnya,NaN,Babownya (Bobovnya),Belarus,NaN,NaN,NaN,NaN,NaN,NaN
1956,FK Mozyrskiy NPZ,NaN,Mazyr (Mozyr),Belarus,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
df_team_data[df_team_data['Address'].isna() & df_team_data['City'].isna()].shape

(75, 10)

In [96]:
df_team_data[df_team_data['Address'].isna() & df_team_data['City'].isna()].head()

,Team Name,Founded,Address,Country,Phone,Name,City,Capacity,E-mail,Fax
23,IF Lödde,NaN,NaN,Sweden,NaN,NaN,NaN,NaN,NaN,NaN
26,FC Loon-Plage,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN
86,Zalaszentgróti VFC,NaN,NaN,Hungary,NaN,NaN,NaN,NaN,NaN,NaN
204,Dvůr Králové,NaN,NaN,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN
283,La Fleche,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# 3 Stop ************************************************************************************

##### Check equal row numbers in 'df_team_country' 'df_team_data'

In [98]:
print('shape - df_team_country - ', df_team_country.shape)
print('shape - df_team_data    - ', df_team_data.shape)

shape - df_team_country -  (1957, 2)
shape - df_team_data    -  (1957, 10)


In [99]:
df_team_country_data = pd.merge(df_team_country, df_team_data, how ='left', left_index=True, right_index=True)
print('shape - ', df_team_country_data.shape)
df_team_country_data.tail(2)

shape -  (1957, 12)


,team,country,Team Name,Founded,Address,Country,Phone,Name,City,Capacity,E-mail,Fax
1955,Bobovnya,Belarus,FK Bobovnya,NaN,Babownya (Bobovnya),Belarus,NaN,NaN,NaN,NaN,NaN,NaN
1956,Mozyr NPZ,Belarus,FK Mozyrskiy NPZ,NaN,Mazyr (Mozyr),Belarus,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df_team_country_data.rename(columns={'Country': 'country_prsng'}, inplace=True)

In [101]:
df_team_country_data.columns = df_team_country_data.columns.str.lower()

In [102]:
df_team_country_data.city = df_team_country_data[['address', 'city']].apply(lambda x: x[0] if pd.isna(x[1]) else x[1], axis=1)

In [103]:
df_team_country_data.city = df_team_country_data.city.map(lambda x: re.sub(',.*', '', str(x)))

In [104]:
df_team_country_data.city = df_team_country_data.city.map(lambda x: re.sub(' \(.*', '', str(x)))

In [105]:
df_team_country_data.city = df_team_country_data.city.map(lambda x: re.sub(' \/.*', '', str(x)))

In [106]:
df_team_country_data.city = df_team_country_data.city.map(lambda x: re.sub('^.*\d* ', '', str(x)))

In [107]:
df_team_country_data.city = df_team_country_data.city.map(lambda x: re.sub(' - .*', '', x))

In [108]:
# 4 Stop **************************************************************

#### Visual check cities names. City with error names need to change it for 'nan', because it parsed in geocoder with central point of country coordinates.

In [109]:
for var in [113, 250, 399, 414, 450, 651, 730, 765, 831, 1009, 1140, 1185, 1310, 1543]:
    df_team_country_data.city.loc[var] = 'nan'

In [110]:
# df_team_country_data.city

In [111]:
# Check 'nan' count and cities name
df_team_country_data[df_team_country_data.city == 'nan'].shape

(88, 12)

In [112]:
df_team_country_data[df_team_country_data.city == 'nan'].tail()

,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1918,Manabita,Ecuador,Club Deportivo Unión Manabita,NaN,NaN,Ecuador,NaN,NaN,nan,NaN,NaN,NaN
1924,Puebla U20,Mexico,Puebla Under 20,NaN,NaN,Mexico,NaN,NaN,nan,NaN,NaN,NaN
1931,Tuv Buganuud,Mongolia,Tuv buganuud FC,NaN,NaN,Mongolia,NaN,NaN,nan,NaN,NaN,NaN
1935,Atletico Samborondon,Ecuador,Club Atlético Samborondón,NaN,NaN,Ecuador,NaN,NaN,nan,NaN,NaN,NaN
1948,Peñarol,Ecuador,Club Deportivo Peñarol,NaN,NaN,Ecuador,NaN,NaN,nan,NaN,NaN,NaN


#### Check the 'City' in names after cut and remove these index from df_team_country_data for manual fix if exist:

In [113]:
df_team_country_data[df_team_country_data.city == 'City'].head(2)

,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
74,United City,Philippines,United City FC,NaN,Bacolod,Philippines,NaN,Panaad Stadium,City,15500,NaN,NaN
599,Davao Aguilas,Philippines,Davao Aguilas FC,NaN,Davao City,Philippines,NaN,Davao del Norte Sports and Tourism Complex,City,3000,NaN,NaN


In [114]:
df_incorrect_city_names_cut_inds = df_team_country_data[df_team_country_data.city == 'City']
print('shape - ', len(df_incorrect_city_names_cut_inds))
df_incorrect_city_names_cut_inds.tail(2)

shape -  4


,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
975,Rayo OKC,USA,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,City,7000,NaN,+1 (405) 232 7223
1511,Bay Cities,USA,Bay Cities FC,NaN,"Redwood City, California",USA,NaN,NaN,City,NaN,NaN,NaN


In [115]:
df_team_country_data = df_team_country_data[~df_team_country_data.index.isin(df_incorrect_city_names_cut_inds.index)].copy()
print('shape - ', df_team_country_data.shape[0])

shape -  1953


##### Some right urls don't parsing data because haven't it.

In [116]:
# teams_with_data_lt[559]

In [117]:
# [(ind, x) for ind, x in enumerate(teams_urls_lt) if x[0] == 'Bush Bucks']

#### Create 'df_wt_cities' and remove rows from 'df_team_country_data' with 'nan' cities

In [118]:
df_wt_cities = df_team_country_data[df_team_country_data.city == 'nan'].copy().reset_index(drop=True)
print('shape - ', df_wt_cities.shape)
df_wt_cities.tail(3)

shape -  (88, 12)


,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
85,Tuv Buganuud,Mongolia,Tuv buganuud FC,NaN,NaN,Mongolia,NaN,NaN,nan,NaN,NaN,NaN
86,Atletico Samborondon,Ecuador,Club Atlético Samborondón,NaN,NaN,Ecuador,NaN,NaN,nan,NaN,NaN,NaN
87,Peñarol,Ecuador,Club Deportivo Peñarol,NaN,NaN,Ecuador,NaN,NaN,nan,NaN,NaN,NaN


In [119]:
print('shape - ', df_team_country_data.shape)
df_team_country_data = df_team_country_data[df_team_country_data.city != 'nan'].copy().reset_index(drop=True)
print('shape - ', df_team_country_data.shape)
df_team_country_data.tail(3)

shape -  (1953, 12)
shape -  (1865, 12)


,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1862,St. Louis City II,USA,St. Louis City SC II,NaN,"St. Louis, Missouri",USA,NaN,NaN,Louis,NaN,NaN,NaN
1863,Bobovnya,Belarus,FK Bobovnya,NaN,Babownya (Bobovnya),Belarus,NaN,NaN,Babownya,NaN,NaN,NaN
1864,Mozyr NPZ,Belarus,FK Mozyrskiy NPZ,NaN,Mazyr (Mozyr),Belarus,NaN,NaN,Mazyr,NaN,NaN,NaN


#### Create 'df_team_wt_data' from unite 'error_urls_names_nd_data_lt' and 'city', 'country' values from 'df_wt_cities'.

In [120]:
wt_cities_teams_lt = df_wt_cities[['team', 'country']].apply(lambda x: (x[0], x[1]), axis=1).tolist()
print('len - ', len(wt_cities_teams_lt))
str(wt_cities_teams_lt[:10])

len -  88


"[('Lodde', 'Sweden'), ('Loon Plage', 'France'), ('Zalaszentgrot', 'Hungary'), ('Rijnvogels', 'Netherlands'), ('Dvur Kralove', 'Czech Republic'), ('Stroitel Kopyl', 'Belarus'), ('La Flèche', 'France'), ('AB Chelghoum Laïd', 'Algeria'), ('TOP Oss Amateurs', 'Netherlands'), ('Pesnica', 'Slovenia')]"

In [121]:
# If error_urls_names_nd_data_lt is empty:
# for_manual_corr_lt =  wt_cities_teams_lt
# Else:
for_manual_corr_lt =  wt_cities_teams_lt + error_urls_names_nd_data_lt
print('len - ', len(for_manual_corr_lt))
str(for_manual_corr_lt[:10])

len -  103


"[('Lodde', 'Sweden'), ('Loon Plage', 'France'), ('Zalaszentgrot', 'Hungary'), ('Rijnvogels', 'Netherlands'), ('Dvur Kralove', 'Czech Republic'), ('Stroitel Kopyl', 'Belarus'), ('La Flèche', 'France'), ('AB Chelghoum Laïd', 'Algeria'), ('TOP Oss Amateurs', 'Netherlands'), ('Pesnica', 'Slovenia')]"

In [122]:
df_team_wt_data         = pd.DataFrame({'team': [x[0] for x in for_manual_corr_lt], 'country': [x[1] for x in for_manual_corr_lt]})
df_team_wt_data['city'] = None
print('shape - ', df_team_wt_data.shape)
df_team_wt_data.tail(2)

shape -  (103, 3)


,team,country,city
101,LASK Amateure,Austria,None
102,Plama Podgrad,Slovenia,None


In [123]:
# If exist df_incorrect_city_names_cut_inds:
merge_columns = [x for x in df_incorrect_city_names_cut_inds.columns if x not in df_team_wt_data.columns]
df_team_wt_data[merge_columns] = np.nan
df_team_wt_data = pd.concat([df_team_wt_data, df_incorrect_city_names_cut_inds])
print('shape - ', df_team_wt_data.shape)
df_team_wt_data.tail(2)

shape -  (107, 12)


,team,country,city,team name,founded,address,country_prsng,phone,name,capacity,e-mail,fax
975,Rayo OKC,USA,City,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,7000,NaN,+1 (405) 232 7223
1511,Bay Cities,USA,City,Bay Cities FC,NaN,"Redwood City, California",USA,NaN,NaN,NaN,NaN,NaN


#### Compare df_tmd_not_exist.shape[0] == df_team_country_data.shape[0] + df_team_wt_data.shape[0]

In [124]:
print('shape - ', df_teams_na_cities.shape[0])
df_team_country_data.shape[0] + df_team_wt_data.shape[0]

shape -  1972


1972

In [125]:
# 5 Stop **********************************************************************************************************************************************

#### Manual fixing teams and countries names in parsing data and add some data

In [126]:
# df_team_wt_data.to_csv('csv_files/df_team_wt_data_bets_coef_w_events.csv', index = None, header=True)

In [127]:
df_after_manual_corr = pd.read_csv('csv_files/bets_games/df_team_wt_data_bets_coef_w_events.csv', dtype=str) 

In [128]:
print('shape - ', df_after_manual_corr.shape)
df_after_manual_corr.tail(2)

shape -  (107, 12)


,team,country,city,team name,founded,address,country_prsng,phone,name,capacity,e-mail,fax
105,Rayo OKC,USA,Oklahoma City,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,7000,NaN,+1 (405) 232 7223
106,Bay Cities,USA,Redwood City,Bay Cities FC,2021,"Redwood City, California",USA,NaN,NaN,3500,NaN,NaN


In [129]:
added_cols = [x for x in df_team_country_data.columns if x not in df_after_manual_corr.columns]
str(added_cols)

'[]'

In [130]:
str(df_team_country_data.columns)

"Index(['team', 'country', 'team name', 'founded', 'address', 'country_prsng',\n       'phone', 'name', 'city', 'capacity', 'e-mail', 'fax'],\n      dtype='object')"

In [131]:
# If don't exist df_incorrect_city_names_cut_inds:
# df_after_manual_corr[[added_cols]] = None

In [132]:
df_team_country_data[df_team_country_data.duplicated(subset=['team', 'country'])]

,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax


In [133]:
df_after_manual_corr[df_after_manual_corr.duplicated(subset=['team', 'country'])]

,team,country,city,team name,founded,address,country_prsng,phone,name,capacity,e-mail,fax


In [134]:
df_team_country_data_full = pd.concat([df_team_country_data, df_after_manual_corr], axis=0, ignore_index=True)

In [135]:
df_team_country_data_full.tail(2)

,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1970,Rayo OKC,USA,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,Oklahoma City,7000,NaN,+1 (405) 232 7223
1971,Bay Cities,USA,Bay Cities FC,2021,"Redwood City, California",USA,NaN,NaN,Redwood City,3500,NaN,NaN


In [136]:
# df_team_country_data_full.shape[0] == df_teams_na_cities.shape[0]
print('shape - df_teams_na_cities        - ', df_teams_na_cities.shape) #df_teams_na_cities.shape)
print('shape - df_team_country_data_full - ', df_team_country_data_full.shape)
df_team_country_data_full.tail(2)

shape - df_teams_na_cities        -  (1972, 2)
shape - df_team_country_data_full -  (1972, 12)


,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1970,Rayo OKC,USA,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,Oklahoma City,7000,NaN,+1 (405) 232 7223
1971,Bay Cities,USA,Bay Cities FC,2021,"Redwood City, California",USA,NaN,NaN,Redwood City,3500,NaN,NaN


In [137]:
df_team_country_data_full[df_team_country_data_full.duplicated(subset=['team', 'country'])]

,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1912,Brda Dobrovo,Slovenia,NaN,1973,NaN,NaN,NaN,NaN,Dobrovo,600,NaN,NaN


In [138]:
df_team_country_data_full.drop_duplicates(subset=['team', 'country'], inplace=True, ignore_index=True)
print('shape - ', df_team_country_data_full.shape)
df_team_country_data_full.tail(2)

shape -  (1971, 12)


,team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
1969,Rayo OKC,USA,Rayo OKC,2015,300 North Walker Avenue 73102 Oklahoma City,USA,+1 (405) 232 7296,Miller Stadium,Oklahoma City,7000,NaN,+1 (405) 232 7223
1970,Bay Cities,USA,Bay Cities FC,2021,"Redwood City, California",USA,NaN,NaN,Redwood City,3500,NaN,NaN


In [139]:
df_team_country_data_full.rename(columns={'team': 'f_team'}, inplace=True)

### Concatinate new and old full teams data.

In [140]:
# Stop 

In [141]:
with open('pickle_files/teams_data/df_full_teams_data_cr_11_10_2022', 'rb') as f:
    df_teams_data = pickle.load(f)

In [142]:
df_teams_data.tail(2)

,f_team,country,team name,founded,address,country_prsng,phone,fax,e-mail,name,city,capacity
14523,Edmonton,USA,FC Edmonton,2010,"9725-62 Avenue T6E 0E4 Edmonton, Alberta",Canada,+1 (780) 700 2600,+1 (780) 439 7557,info@fcedmonton.com,Clarke Field,"Edmonton, Alberta",5000
14524,Calgary,USA,Calgary Foothills Soccer Club,1972,"104, 3916 56th Avenue SE AB T2C 2B5 Calgary",Canada,NaN,NaN,NaN,Mount Royal University,"Calgary, Alberta",2000


In [143]:
df_teams_data.city = df_teams_data[['address', 'city']].apply(lambda x: x[0] if str(x[1]) == 'nan' else x[1], axis=1)

In [144]:
index1 = pd.MultiIndex.from_arrays([df_teams_data[col] for col in ['f_team', 'country']])
index2 = pd.MultiIndex.from_arrays([df_team_country_data_full[col] for col in ['f_team', 'country']])
dubl_index_w_data = df_teams_data.loc[index1.isin(index2)].index
print('len - ', len(dubl_index_w_data))
dubl_index_w_data[:3]

len -  102


Int64Index([9252, 9253, 9259], dtype='int64')

In [145]:
df_teams_data = df_teams_data[~df_teams_data.index.isin(dubl_index_w_data)].copy()
df_teams_data.shape

(14423, 12)

In [146]:
df_teams_data[df_teams_data.city.isna()].count()

f_team           297
country          297
team name        297
founded            7
address            0
country_prsng    295
phone              1
fax                1
e-mail             4
name               0
city               0
capacity           0
dtype: int64

In [147]:
df_teams_data_concat = pd.concat([df_team_country_data_full, df_teams_data], ignore_index=True)
print('shape - ', df_teams_data_concat.shape)
df_teams_data_concat.tail(2)

shape -  (16394, 12)


,f_team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
16392,Edmonton,USA,FC Edmonton,2010,"9725-62 Avenue T6E 0E4 Edmonton, Alberta",Canada,+1 (780) 700 2600,Clarke Field,"Edmonton, Alberta",5000,info@fcedmonton.com,+1 (780) 439 7557
16393,Calgary,USA,Calgary Foothills Soccer Club,1972,"104, 3916 56th Avenue SE AB T2C 2B5 Calgary",Canada,NaN,Mount Royal University,"Calgary, Alberta",2000,NaN,NaN


In [148]:
# Fix cities with number and one ar two upper letters in name. ***************************************************

In [149]:
df_teams_data_concat[df_teams_data_concat.duplicated(subset=['f_team', 'country'])]

,f_team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax


In [150]:
df_full_tmd_city_na = df_teams_data_concat[df_teams_data_concat.city.isna()].reset_index(drop=True).copy()
print('shape - ', df_full_tmd_city_na.shape)
df_full_tmd_city_na.tail(2)

shape -  (297, 12)


,f_team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
295,Brunei DS,Malaysia,Brunei DS,NaN,NaN,Malaysia,NaN,NaN,NaN,NaN,NaN,NaN
296,Selangor MKL,Malaysia,Selangor MK Land,NaN,NaN,Malaysia,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
# df_full_tmd_city_na.to_csv('csv_files/bets_games/df_full_tmd_city_na_23_10_2022.csv', index = None, header=True)

In [152]:
# df_full_tmd_city_na_am_corr = pd.read_csv('csv_files/bets_games/df_full_tmd_city_na_23_10_2022.csv', dtype=str) 

In [153]:
df_teams_data_concat = df_teams_data_concat[~df_teams_data_concat.city.isna()].reset_index(drop=True).copy()
print('shape - ', df_teams_data_concat.shape)
df_teams_data_concat.tail(2)

shape -  (16097, 12)


,f_team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax
16095,Edmonton,USA,FC Edmonton,2010,"9725-62 Avenue T6E 0E4 Edmonton, Alberta",Canada,+1 (780) 700 2600,Clarke Field,"Edmonton, Alberta",5000,info@fcedmonton.com,+1 (780) 439 7557
16096,Calgary,USA,Calgary Foothills Soccer Club,1972,"104, 3916 56th Avenue SE AB T2C 2B5 Calgary",Canada,NaN,Mount Royal University,"Calgary, Alberta",2000,NaN,NaN


In [227]:
df_teams_data_concat.city = df_teams_data_concat.city.str.strip()

In [228]:
df_teams_data_concat.city.str.findall(r'^[0-9]\d{1}?').value_counts()

[]      16092
[12]        1
[84]        1
[10]        1
[35]        1
[13]        1
Name: city, dtype: int64

In [217]:
# df_teams_data_concat.city.str.findall(r'[0-9].*').value_counts()

In [224]:
# df_teams_data_concat.city.str.findall(r'^[A-Z]*').value_counts()

In [229]:
df_teams_data_concat.city.str.findall(r'^[^A-Z]*').value_counts()

[]                         15674
[al-]                         56
[İstanbul]                    46
[ad-]                         20
[ʿ]                           14
[Örebro]                       8
[İzmir]                        7
[ẓabī]                         7
[nd]                           7
[d']                           6
[Ålesund]                      5
[Ħamrun]                       5
[Århus]                        4
[Šiauliai]                     4
[Şımkent]                      4
[Čačak]                        4
[as-]                          3
[Şımkent (]                    3
[Štip]                         3
[Łódź]                         3
[Ḥalab]                        3
[Águilas]                      3
[İmişli]                       3
[Český ]                       3
[Šumen (]                      2
[Şimleu ]                      2
[vul. ]                        2
[Świnoujście]                  2
[le-]                          2
[Šabac]                        2
[Ōsaka]   

In [212]:
df_teams_data_concat[df_teams_data_concat.city == 13]

,f_team,country,team name,founded,address,country_prsng,phone,name,city,capacity,e-mail,fax


In [154]:
6 Stop *************************************************************************************************

SyntaxError: invalid syntax (658129006.py, line 1)

In [ ]:
# file = open('pickle_files/teams_data/df_full_teams_data_cr_11_10_2022', 'wb')
# pickle.dump(df_team_country_data_full, file)  
# file.close()

### Go to the 'teams_data.ipynb' - new solution with work with 'full_data' file.

#### 'Soccer365' work after manual correct for 

In [ ]:
# colnames = ['team_id', 'f_team', 'country', 'tm_data', 'city']

In [ ]:
# df_after_corr = pd.read_csv('csv_files/df_teams_data_26_05_2022_four.csv', names=colnames, header=None, dtype=str)

In [ ]:
# print('shape - ', df_after_corr.shape[0])
# df_after_corr.head(3)

In [ ]:
# df_after_corr.drop_duplicates(subset=['f_team', 'country'], inplace=True)
# df_after_corr.reset_index(drop=True, inplace=True)

In [ ]:
# df_after_corr.drop(columns='tm_data', inplace=True)

In [ ]:
# df_after_corr.tail(3)

In [ ]:
# file = open('pickle_files/teams_data/df_after_corr_26_05_2021_four', 'wb')
# pickle.dump(df_after_corr, file)  
# file.close()